In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('Fake_Real_Data.csv')
df.head(5)

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [3]:
df.shape

(9900, 2)

In [4]:
df.label.value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [5]:
df['label_num']=df['label'].map({'Fake':0,'Real':1})
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [6]:
import spacy
nlp=spacy.load('en_core_web_lg')

In [7]:
doc=nlp('Top Trump Surrogate BRUTALLY Stabs Him In The Back')
doc.vector

array([-5.7353568e-01, -9.0108120e-01,  4.9225557e-01,  1.2843722e+00,
        3.3725412e+00, -9.6783000e-01, -5.8240998e-01,  5.8662839e-02,
        2.3800224e-01, -2.2299123e+00, -3.6770555e-01, -3.6848328e-01,
       -1.2570310e+00, -4.5659563e-01,  5.2825904e-01,  8.2203335e-01,
       -2.4962442e-01,  1.1932555e+00,  8.8347107e-01,  1.5664444e+00,
       -9.0263009e-01,  8.6768448e-01,  6.6950548e-01,  1.1872389e+00,
        5.0095573e-02,  1.9816178e+00, -4.1208947e-01,  1.2451012e+00,
       -1.7988542e+00,  3.0229831e+00,  1.7740923e+00,  7.9722327e-01,
       -5.5664670e-01,  2.3080637e+00, -3.4971132e+00, -5.4909256e-03,
       -1.2592912e+00, -8.2379657e-01, -6.2228000e-01, -1.7302179e+00,
        2.6951568e+00, -2.5824645e-01,  4.1109005e-01,  2.3796468e+00,
       -7.8466678e-01, -1.5188489e+00, -4.1179266e+00,  3.6906698e-01,
        9.4939339e-01, -5.6610328e-01, -2.2244444e+00,  1.2300978e+00,
        5.5450865e-04, -1.5626302e+00,  7.8238225e-01,  1.6799229e+00,
      

In [8]:
doc.vector.shape

(300,)

In [9]:
df['vector']=df['Text'].apply(lambda text: nlp(text).vector)
df.head()

,Text,label,label_num,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[-0.6759837, 1.4263071, -2.318466, -0.451093, ..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[-1.8355803, 1.3101058, -2.4919677, 1.0268308,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[-1.9851209, 0.14389805, -2.4221718, 0.9133005..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-2.7812982, -0.16120885, -1.609772, 1.3624227..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[-2.2010763, 0.9961637, -2.4088492, 1.128273, ..."


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(
    df.vector.values,
    df.label_num,
    test_size=0.2,
    random_state=42
)

In [11]:
import numpy as np
X_train_2d=np.stack(X_train)
X_test_2d=np.stack(X_test)

In [20]:
X_train_2d

array([[-2.0152547 ,  0.98641217, -2.3584037 , ..., -1.0675061 ,
        -1.9569906 ,  0.93659127],
       [-1.0012243 ,  1.4828775 , -2.1455953 , ...,  0.3300781 ,
        -1.799691  ,  0.7323975 ],
       [-1.9412339 ,  1.0061342 , -1.2211968 , ..., -1.0381267 ,
        -1.6678015 ,  1.0008049 ],
       ...,
       [-1.1860418 ,  0.9153    , -2.4448311 , ..., -0.32731298,
        -2.97908   ,  1.1330711 ],
       [-2.0719678 ,  1.015092  , -2.2288282 , ..., -0.98319465,
        -1.6227347 ,  0.66743565],
       [-1.7736759 ,  0.6012506 , -1.835393  , ..., -0.8052035 ,
        -2.0403569 ,  0.6292755 ]], dtype=float32)

In [13]:
#in this below code there will we an error -> Negative values in data passed to MultinomialNB 
#the error occurs bcz there is negative values in X_train_2d and X_test_2d
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(X_train_2d,y_train) 

ValueError: Negative values in data passed to MultinomialNB (input X)

In [14]:
#we use minmaxscaler for alter data to zero mean
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler_train_embed=scaler.fit_transform(X_train_2d)
scaler_test_embed=scaler.transform(X_test_2d)
clf=MultinomialNB()
clf.fit(scaler_train_embed,y_train)

MultinomialNB()

In [15]:
from sklearn.metrics import classification_report

In [18]:
y_pred=clf.predict(scaler_test_embed)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95       973
           1       0.94      0.96      0.95      1007

    accuracy                           0.95      1980
   macro avg       0.95      0.95      0.95      1980
weighted avg       0.95      0.95      0.95      1980



In [22]:
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier(n_neighbors=5, metric='euclidean')
clf.fit(X_train_2d,y_train)
y_pred=clf.predict(X_test_2d)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       973
           1       0.99      1.00      0.99      1007

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

